## 라이브러리 import

In [ ]:
import torch
import numpy as np
import random
import json
from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation
from PIL import Image
from transformers import AutoFeatureExtractor
import evaluate

## 시드 고정

In [ ]:
torch.cuda.manual_seed(42)
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


## 데이터셋 경로 지정

In [ ]:
import datasets
import glob
from sklearn.model_selection import train_test_split
IMAGES = sorted(glob.glob("D:/baselinegu/data/train/x/*.png"))
SEG_MAPS = sorted(glob.glob("D:/baselinegu/data/train/y/*.png"))
up_IMAGES = sorted(glob.glob("D:/baselinegu/data/up/x/*.png"))
up_SEG = sorted(glob.glob("D:/baselinegu/data/up/y/*.png"))
train_IMAGES = IMAGES
#val_IMAGES = IMAGES[10000:]
train_SEGMAPS = SEG_MAPS
#val_SEGMAPS = SEG_MAPS[10000:]
test_IMAGES = glob.glob("D:/baselinegu/data/test/x/*.png")
test_SEGMAPS = sorted(glob.glob("D:/baselinegu/data/test/Sample_Submission/*.png"))
cut_IMAGES = sorted(glob.glob("D:/baselinegu/data/cut/x/*.png"))
cut_SEGMAPS = sorted(glob.glob("D:/baselinegu/data/cut/y/*.png"))
#train_IMAGES += up_IMAGES
#train_SEGMAPS += up_SEG
print(len(train_IMAGES))
print(len(train_SEGMAPS))

#print(len(val_IMAGES))
#print(len(val_SEGMAPS))

print(len(cut_IMAGES))
print(len(cut_SEGMAPS))

12000
12000
3203
3203


In [ ]:
train_IMAGES, val_IMAGES = train_test_split(train_IMAGES, test_size=0.3, random_state=42, shuffle=True)
print(len(train_IMAGES))
print(len(val_IMAGES))

8400
3600


In [ ]:
train_IMAGES += up_IMAGES
train_IMAGES += cut_IMAGES


In [ ]:
print(len(train_IMAGES))

17559


## 데이터 로더 지정

In [ ]:
train_ds = datasets.Dataset.from_dict(
            {"image": train_IMAGES, "label": list(map(lambda x : x.replace('x', 'y'),train_IMAGES))}, 
            features=datasets.Features({"image": datasets.Image(), "label": datasets.Image()}),
            split="train")
val_ds = datasets.Dataset.from_dict(
            {"image": val_IMAGES, "label": list(map(lambda x : x.replace('x', 'y'),val_IMAGES))}, 
            features=datasets.Features({"image": datasets.Image(), "label": datasets.Image()}),
            split="val")

In [ ]:
print(train_ds)
print(val_ds)


Dataset({
    features: ['image', 'label'],
    num_rows: 17559
})
Dataset({
    features: ['image', 'label'],
    num_rows: 3600
})


## 데이터 전처리

In [ ]:
feature_extractor = SegformerFeatureExtractor.from_pretrained("nvidia/segformer-b4-finetuned-cityscapes-1024-1024",reduce_labels=False)
def train_transforms(example_batch):
  images = [x for x in example_batch["image"]]
  labels = [x for x in example_batch["label"]]
  inputs = feature_extractor(images,labels,return_tensors="pt",padding=True)
  return inputs

def val_transforms(example_batch):
    images = [x for x in example_batch["image"]]
    labels = [x for x in example_batch["label"]]
    inputs = feature_extractor(images, labels,return_tensors="pt",padding=True)
    return inputs

def test_transforms(example_batch):
    images = [x for x in example_batch["image"]]
    labels = [x for x in example_batch["label"]]
    inputs = feature_extractor(images, labels,return_tensors="pt",padding=True)
    return inputs


train_ds.set_transform(train_transforms)
val_ds.set_transform(val_transforms)


## 라벨수 지정

In [ ]:
num_labels=4

## 모델 지정 및 gpu활용

In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
model = SegformerForSemanticSegmentation.from_pretrained('nvidia/segformer-b4-finetuned-cityscapes-1024-1024',num_labels=4,ignore_mismatched_sizes=True).to(device)

Downloading:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

C:\Users\taemi\.conda\envs\hug\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\taemi\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

In [ ]:
model.config

SegformerConfig {
  "_name_or_path": "taemin6697/4batch_18000iter_backbone",
  "architectures": [
    "SegformerForSemanticSegmentation"
  ],
  "attention_probs_dropout_prob": 0.0,
  "classifier_dropout_prob": 0.1,
  "decoder_hidden_size": 768,
  "depths": [
    3,
    8,
    27,
    3
  ],
  "downsampling_rates": [
    1,
    4,
    8,
    16
  ],
  "drop_path_rate": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_sizes": [
    64,
    128,
    320,
    512
  ],
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-06,
  "mlp_ratios": [
    4,
    4,
    4,
    4
  ],
  "model_type": "segformer",
  "num_attention_heads": [
    1,
    2,
    5,
    8
  ],
  "num_channels": 3,
  "num_encoder_blocks": 4,
  "patch_sizes": [
    7,
    3,
    3,
    3
  ],
  "re

## 훈련 세팅 설정

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = "b4_pretrained_batch_4",
    learning_rate=6e-6,
    num_train_epochs=50,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_total_limit=50,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=1500,
    eval_steps=1500,
    logging_steps=1,
    eval_accumulation_steps=10,
    remove_unused_columns=False,

)

In [ ]:
metric = evaluate.load("mean_iou")

In [ ]:
import torch
import numpy as np 
from torch import nn
def compute_metrics(eval_pred):
    with torch.no_grad():
        logits, labels = eval_pred
        
        logits_tensor = torch.from_numpy(logits)
        logits_tensor = nn.functional.interpolate(
            logits_tensor,
            size=labels.shape[-2:],
            mode="bilinear",
            align_corners=False,
        ).argmax(dim=1)
        pred_labels = logits_tensor.detach().cpu().numpy()
        print('max=',np.max(pred_labels))
        print('min=',np.min(pred_labels))

        metrics = metric._compute(
            predictions=pred_labels,
            references=labels,
            num_labels=num_labels,
            ignore_index=255,
            reduce_labels=False,
        )
        for key, value in metrics.items():
            if type(value) is np.ndarray:
                metrics[key] = value.tolist()
        return metrics

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
)

## 훈련

In [ ]:
trainer.train()

C:\Users\taemi\.conda\envs\hug\lib\site-packages\transformers\optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 17559
  Num Epochs = 50
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 219500
  Number of trainable parameters = 63996100


Step,Training Loss,Validation Loss,Mean Iou,Mean Accuracy,Overall Accuracy,Per Category Iou,Per Category Accuracy
2200,0.019500,0.034577,0.833693,0.907595,0.988876,"[0.9890527313958374, 0.7907119051062578, 0.8570974396574677, 0.6979099613123291]","[0.9960218492991216, 0.917199543533476, 0.9415550972018024, 0.7756018215980222]"
4400,0.015300,0.028073,0.864399,0.926748,0.990676,"[0.9906498679280075, 0.8487305283577348, 0.8735459716289549, 0.7446697296715641]","[0.996375486567416, 0.9421293170219411, 0.9488605557891203, 0.8196277334496205]"
6600,0.044700,0.024871,0.879383,0.939248,0.991776,"[0.9916984091610799, 0.8681535664830972, 0.8806307230673817, 0.7770481922116391]","[0.9962516890882793, 0.948793700427833, 0.953276575719007, 0.8586702508756506]"
8800,0.017200,0.020858,0.893877,0.945491,0.992970,"[0.9928736095116244, 0.88000947028494, 0.8946979806409187, 0.8079264497872868]","[0.9968810064239635, 0.9504434955608677, 0.9570483956293127, 0.8775894788789844]"
11000,0.027100,0.019252,0.902511,0.953540,0.993598,"[0.9934813724505704, 0.8900975741955824, 0.9000381068222759, 0.8264285903051621]","[0.9968212127467554, 0.9564391434430737, 0.9618831867085822, 0.8990178710689908]"
13200,0.031400,0.016651,0.908329,0.954576,0.994281,"[0.9941761995428036, 0.8919775201585535, 0.9020146891934476, 0.8451477615739804]","[0.9972574543130969, 0.9529309503887344, 0.959652345900033, 0.9084613887446195]"
15400,0.005600,0.015423,0.912741,0.958125,0.994624,"[0.9945385656156244, 0.8949677625372069, 0.9073360271015677, 0.8541213513659778]","[0.99737353928402, 0.9572830681827147, 0.9626203555687924, 0.9152214239428562]"
17600,0.014000,0.014148,0.919357,0.959582,0.994989,"[0.9948620175731528, 0.9075648091519007, 0.9110109598658294, 0.8639884943217992]","[0.9975511955403664, 0.9582692326433895, 0.9608457202658086, 0.921660657254463]"
19800,0.025700,0.013403,0.922093,0.959731,0.995200,"[0.9950742022478101, 0.9129664110715299, 0.9114791229311517, 0.8688529911784332]","[0.9978155363385088, 0.9593368426865811, 0.9618025588644967, 0.9199679774488364]"
22000,0.009700,0.012456,0.925771,0.962580,0.995482,"[0.9953693568650199, 0.9171835452457759, 0.9139287678307938, 0.8766024598996899]","[0.9978590189468974, 0.9580615580099062, 0.9666879480295953, 0.9277102751283858]"


***** Running Evaluation *****
  Num examples = 3600
  Batch size = 4


max= 3
min= 0


Saving model checkpoint to b4-18000_pretrained_batch_4\checkpoint-2200
Configuration saved in b4-18000_pretrained_batch_4\checkpoint-2200\config.json
Model weights saved in b4-18000_pretrained_batch_4\checkpoint-2200\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3600
  Batch size = 4


...

max= 3
min= 0


Saving model checkpoint to b4-18000_pretrained_batch_4\checkpoint-50600
Configuration saved in b4-18000_pretrained_batch_4\checkpoint-50600\config.json
Model weights saved in b4-18000_pretrained_batch_4\checkpoint-50600\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3600
  Batch size = 4


max= 3
min= 0


Saving model checkpoint to b4-18000_pretrained_batch_4\checkpoint-52800
Configuration saved in b4-18000_pretrained_batch_4\checkpoint-52800\config.json
Model weights saved in b4-18000_pretrained_batch_4\checkpoint-52800\pytorch_model.bin


KeyboardInterrupt: 